# Refinement of the Step Size of the Numerical Scheme

_Kevin Siswandi | May 2020_

In Multistep Neural Network, training is done to find the optimal parameters via minimising loss function:

$$ \arg \min_w \frac{1}{N - M + 1} \sum_{n = M}^{N} |\textbf{y}_n|^2 $$

where N is the number of data points and M multi steps. The linear difference/residual operator is defined as

$$ \textbf{y}_n = \sum_{n=0}^M \left(\alpha_m x_{n-m} + h \beta_m \textbf{f}(\textbf{x}_{n-m}) \right)$$

for $n=M,...,N$. Now, which step size to use is problem-dependent and hence needs to be tuned for performance.

In [1]:
import numpy as np
from model import lmmNet
from train import *
from predict import *
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import timeit

In [2]:
# specify the settings for the bier data

t0, T, h = 0, 500, 0.2 #seconds
x0 = np.array([4, 3]) #initial conditions: ATP = 4, G = 3 -- default Bier model
params = {'Vin': 0.36, 'k1': 0.02, 'kp':4, 'km':15} # damped oscillation
f_bier = lambda x, t: bier(x, t, params)

To optimize for the step size, we perform a grid search on candidate step sizes.

In [3]:
h_list = [0.3, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2] # list of step sizes to try

hidden_layer_units = 256 # number of units for the hidden layer
M = 1 # number of steps
scheme = 'AM' # LMM scheme

time_points, data = create_training_data(t0, T, h, f_bier, x0)
data.shape

TensorShape([1, 2500, 2])

In [4]:
# test if it works for 5 epochs
net = lmmNet(h, data, M, scheme, hidden_layer_units)
N_Iter = 5
net.train(N_Iter)
pred = odeint(predict_fn, x0, time_points, args=(net,))

plt.figure(figsize=(20, 10))

for species in range(2):
    plt.subplot(2,1,species + 1)
    plt.plot(time_points, data[0, :, species], 'r-', label='Simulated data')
    plt.plot(time_points, pred[:,species], 'k--', label='Predicted dynamics')
    plt.xlabel('t')
    plt.ylabel('S_' + str(species))
    plt.legend()
    
plt.title('Predicted Trajectories of 2D Yeast Glycolytic Oscillator')


NameError: name 'tf' is not defined

In [5]:
for h in h_list:
    net = lmmNet(h, data, M, scheme, hidden_layer_units)
    N_Iter = 10000
    net.train(N_Iter)
    pred = odeint(predict_fn, x0, time_points, args=(net,))

(1000, 7)


In [6]:
step_size = 0.01

net = lmmNet(step_size, gly_data.reshape(1, gly_data.shape[0], -1), M = 1, scheme='AM') # use trapezoidal rule (smallest error constant that is also stable)

In [7]:
epochs = 50000
net.train(epochs)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 0, Loss: 9.688e-03, Time: 0.16
Epoch: 10, Loss: 9.408e-03, Time: 0.82
Epoch: 20, Loss: 9.159e-03, Time: 1.57
Epoch: 30, Loss: 8.906e-03, Time: 2.28
Epoch: 40, Loss: 8.642e-03, Time: 2.88
Epoch: 50, Loss: 8.365e-03, Time: 3.45
Epoch: 60, Loss: 8.072e-03, Time: 4.01
Epoch: 70, Loss: 7.766e-03, Time: 4.57
Epoch: 80, Loss: 7.453e-03, Time: 5.14
Epoch: 90, Loss: 7.138e-03, Time: 5.70
Epoch: 100, Loss: 6.829e-03, Time: 6.27
Epoch: 110, Loss: 6.534e-03, Time: 6.82
Epoch: 120, Loss: 6.261e-03, Time: 7.38

Epoch: 1820, Loss: 2.521e-04, Time: 98.87
Epoch: 1830, Loss: 2.487e-04, Time: 99.41
Epoch: 1840, Loss: 2.453e-04, Time: 99.94
Epoch: 1850, Loss: 2.420e-04, Time: 100.46
Epoch: 1860, Loss: 2.388e-04, Time: 100.99
Epoch: 1870, Loss: 2.356e-04, Time: 101.51
Epoch: 1880, Loss: 2.325e-04, Time: 102.36
Epoch: 1890, Loss: 2.294e-04, Time: 103.26
Epoch: 1900, Loss: 2.264e-04, Time: 103.79
Epoch: 1910, Loss: 2.235e-04, Time: 104.32
Epoch: 1920, Loss: 2.206e-04, Time: 104.85
Epoch: 1930, Loss: 2.178e-04, Time: 105.38
Epoch: 1940, Loss: 2.150e-04, Time: 105.91
Epoch: 1950, Loss: 2.123e-04, Time: 106.43
Epoch: 1960, Loss: 2.096e-04, Time: 107.00
Epoch: 1970, Loss: 2.070e-04, Time: 107.52
Epoch: 1980, Loss: 2.044e-04, Time: 108.05
Epoch: 1990, Loss: 2.018e-04, Time: 108.67
Epoch: 2000, Loss: 1.993e-04, Time: 109.67
Epoch: 2010, Loss: 1.969e-04, Time: 110.38
Epoch: 2020, Loss: 1.945e-04, Time: 111.13
Epoch: 2030, Loss: 1.921e-04, Time: 112.03
Epoch: 2040, Loss: 1.898e-04, Time: 112.81
Epoch: 2050, L

Epoch: 3730, Loss: 5.164e-05, Time: 234.74
Epoch: 3740, Loss: 5.138e-05, Time: 235.39
Epoch: 3750, Loss: 5.116e-05, Time: 235.96
Epoch: 3760, Loss: 5.094e-05, Time: 236.59
Epoch: 3770, Loss: 5.073e-05, Time: 237.20
Epoch: 3780, Loss: 5.053e-05, Time: 237.76
Epoch: 3790, Loss: 5.033e-05, Time: 238.27
Epoch: 3800, Loss: 5.014e-05, Time: 238.98
Epoch: 3810, Loss: 4.994e-05, Time: 239.71
Epoch: 3820, Loss: 4.975e-05, Time: 240.28
Epoch: 3830, Loss: 4.955e-05, Time: 240.79
Epoch: 3840, Loss: 4.936e-05, Time: 241.31
Epoch: 3850, Loss: 4.917e-05, Time: 241.85
Epoch: 3860, Loss: 4.899e-05, Time: 242.47
Epoch: 3870, Loss: 4.880e-05, Time: 243.41
Epoch: 3880, Loss: 4.861e-05, Time: 243.97
Epoch: 3890, Loss: 4.843e-05, Time: 244.51
Epoch: 3900, Loss: 4.829e-05, Time: 245.06
Epoch: 3910, Loss: 4.809e-05, Time: 245.60
Epoch: 3920, Loss: 4.789e-05, Time: 246.16
Epoch: 3930, Loss: 4.771e-05, Time: 246.70
Epoch: 3940, Loss: 4.753e-05, Time: 247.25
Epoch: 3950, Loss: 4.736e-05, Time: 247.79
Epoch: 3960

Epoch: 5640, Loss: 2.967e-05, Time: 338.66
Epoch: 5650, Loss: 2.960e-05, Time: 339.18
Epoch: 5660, Loss: 2.953e-05, Time: 339.70
Epoch: 5670, Loss: 2.946e-05, Time: 340.21
Epoch: 5680, Loss: 2.943e-05, Time: 340.72
Epoch: 5690, Loss: 2.949e-05, Time: 341.24
Epoch: 5700, Loss: 2.926e-05, Time: 341.75
Epoch: 5710, Loss: 2.917e-05, Time: 342.27
Epoch: 5720, Loss: 2.910e-05, Time: 342.79
Epoch: 5730, Loss: 2.903e-05, Time: 343.31
Epoch: 5740, Loss: 2.896e-05, Time: 343.82
Epoch: 5750, Loss: 2.889e-05, Time: 344.34
Epoch: 5760, Loss: 2.884e-05, Time: 344.86
Epoch: 5770, Loss: 2.899e-05, Time: 345.38
Epoch: 5780, Loss: 2.869e-05, Time: 345.89
Epoch: 5790, Loss: 2.861e-05, Time: 346.40
Epoch: 5800, Loss: 2.854e-05, Time: 346.92
Epoch: 5810, Loss: 2.847e-05, Time: 347.44
Epoch: 5820, Loss: 2.840e-05, Time: 347.95
Epoch: 5830, Loss: 2.833e-05, Time: 348.46
Epoch: 5840, Loss: 2.827e-05, Time: 349.00
Epoch: 5850, Loss: 2.822e-05, Time: 349.51
Epoch: 5860, Loss: 2.818e-05, Time: 350.04
Epoch: 5870

Epoch: 7550, Loss: 1.905e-05, Time: 441.07
Epoch: 7560, Loss: 1.900e-05, Time: 441.60
Epoch: 7570, Loss: 1.896e-05, Time: 442.11
Epoch: 7580, Loss: 1.892e-05, Time: 442.62
Epoch: 7590, Loss: 1.888e-05, Time: 443.13
Epoch: 7600, Loss: 1.884e-05, Time: 443.64
Epoch: 7610, Loss: 1.887e-05, Time: 444.14
Epoch: 7620, Loss: 1.906e-05, Time: 444.71
Epoch: 7630, Loss: 1.878e-05, Time: 445.37
Epoch: 7640, Loss: 1.868e-05, Time: 445.88
Epoch: 7650, Loss: 1.863e-05, Time: 446.39
Epoch: 7660, Loss: 1.859e-05, Time: 446.90
Epoch: 7670, Loss: 1.854e-05, Time: 447.41
Epoch: 7680, Loss: 1.850e-05, Time: 447.91
Epoch: 7690, Loss: 1.846e-05, Time: 448.42
Epoch: 7700, Loss: 1.842e-05, Time: 448.93
Epoch: 7710, Loss: 1.838e-05, Time: 449.43
Epoch: 7720, Loss: 1.834e-05, Time: 449.94
Epoch: 7730, Loss: 1.830e-05, Time: 450.45
Epoch: 7740, Loss: 1.844e-05, Time: 450.95
Epoch: 7750, Loss: 1.849e-05, Time: 451.46
Epoch: 7760, Loss: 1.825e-05, Time: 452.02
Epoch: 7770, Loss: 1.814e-05, Time: 452.53
Epoch: 7780

...

Epoch: 42580, Loss: 7.928e-07, Time: 2370.93
Epoch: 42590, Loss: 7.916e-07, Time: 2371.64
Epoch: 42600, Loss: 7.910e-07, Time: 2372.34
Epoch: 42610, Loss: 7.907e-07, Time: 2373.03
Epoch: 42620, Loss: 7.903e-07, Time: 2373.72
Epoch: 42630, Loss: 7.900e-07, Time: 2374.80
Epoch: 42640, Loss: 7.897e-07, Time: 2375.84
Epoch: 42650, Loss: 7.917e-07, Time: 2376.48
Epoch: 42660, Loss: 8.925e-07, Time: 2377.18
Epoch: 42670, Loss: 1.023e-06, Time: 2377.87
Epoch: 42680, Loss: 8.287e-07, Time: 2378.56
Epoch: 42690, Loss: 8.392e-07, Time: 2379.25
Epoch: 42700, Loss: 7.918e-07, Time: 2380.13
Epoch: 42710, Loss: 7.953e-07, Time: 2381.20
Epoch: 42720, Loss: 7.892e-07, Time: 2382.11
Epoch: 42730, Loss: 7.986e-07, Time: 2382.98
Epoch: 42740, Loss: 9.258e-07, Time: 2383.86
Epoch: 42750, Loss: 8.292e-07, Time: 2384.95
Epoch: 42760, Loss: 8.131e-07, Time: 2386.10
Epoch: 42770, Loss: 8.007e-07, Time: 2386.78
Epoch: 42780, Loss: 7.932e-07, Time: 2387.52
Epoch: 42790, Loss: 7.869e-07, Time: 2388.20
Epoch: 428

Epoch: 44410, Loss: 7.363e-07, Time: 2509.03
Epoch: 44420, Loss: 7.520e-07, Time: 2509.70
Epoch: 44430, Loss: 7.420e-07, Time: 2510.51
Epoch: 44440, Loss: 7.334e-07, Time: 2511.18
Epoch: 44450, Loss: 7.311e-07, Time: 2511.90
Epoch: 44460, Loss: 7.303e-07, Time: 2512.59
Epoch: 44470, Loss: 7.298e-07, Time: 2513.29
Epoch: 44480, Loss: 7.294e-07, Time: 2513.97
Epoch: 44490, Loss: 7.292e-07, Time: 2514.80
Epoch: 44500, Loss: 7.343e-07, Time: 2516.07
Epoch: 44510, Loss: 1.023e-06, Time: 2516.77
Epoch: 44520, Loss: 8.099e-07, Time: 2517.51
Epoch: 44530, Loss: 8.509e-07, Time: 2518.27
Epoch: 44540, Loss: 7.300e-07, Time: 2518.94
Epoch: 44550, Loss: 7.435e-07, Time: 2519.79
Epoch: 44560, Loss: 7.284e-07, Time: 2520.45
Epoch: 44570, Loss: 7.275e-07, Time: 2521.15
Epoch: 44580, Loss: 7.276e-07, Time: 2521.88
Epoch: 44590, Loss: 7.319e-07, Time: 2522.58
Epoch: 44600, Loss: 8.526e-07, Time: 2523.27
Epoch: 44610, Loss: 7.499e-07, Time: 2523.96
Epoch: 44620, Loss: 7.281e-07, Time: 2524.64
Epoch: 446

Epoch: 46240, Loss: 6.964e-07, Time: 2676.58
Epoch: 46250, Loss: 6.842e-07, Time: 2677.22
Epoch: 46260, Loss: 6.946e-07, Time: 2677.83
Epoch: 46270, Loss: 8.435e-07, Time: 2678.50
Epoch: 46280, Loss: 6.794e-07, Time: 2679.26
Epoch: 46290, Loss: 6.830e-07, Time: 2680.30
Epoch: 46300, Loss: 6.842e-07, Time: 2680.95
Epoch: 46310, Loss: 6.823e-07, Time: 2681.68
Epoch: 46320, Loss: 6.786e-07, Time: 2682.36
Epoch: 46330, Loss: 6.781e-07, Time: 2683.07
Epoch: 46340, Loss: 6.773e-07, Time: 2683.76
Epoch: 46350, Loss: 6.768e-07, Time: 2684.82
Epoch: 46360, Loss: 6.770e-07, Time: 2685.59
Epoch: 46370, Loss: 6.912e-07, Time: 2686.44
Epoch: 46380, Loss: 1.221e-06, Time: 2687.22
Epoch: 46390, Loss: 9.140e-07, Time: 2687.90
Epoch: 46400, Loss: 7.483e-07, Time: 2688.63
Epoch: 46410, Loss: 6.801e-07, Time: 2689.29
Epoch: 46420, Loss: 6.891e-07, Time: 2689.89
Epoch: 46430, Loss: 6.766e-07, Time: 2691.06
Epoch: 46440, Loss: 6.745e-07, Time: 2691.79
Epoch: 46450, Loss: 6.745e-07, Time: 2692.51
Epoch: 464

Epoch: 48070, Loss: 6.752e-07, Time: 2816.91
Epoch: 48080, Loss: 6.516e-07, Time: 2817.60
Epoch: 48090, Loss: 6.357e-07, Time: 2818.29
Epoch: 48100, Loss: 6.360e-07, Time: 2818.95
Epoch: 48110, Loss: 6.345e-07, Time: 2819.63
Epoch: 48120, Loss: 6.352e-07, Time: 2820.29
Epoch: 48130, Loss: 6.505e-07, Time: 2820.98
Epoch: 48140, Loss: 1.386e-06, Time: 2821.66
Epoch: 48150, Loss: 9.618e-07, Time: 2822.34
Epoch: 48160, Loss: 6.762e-07, Time: 2823.02
Epoch: 48170, Loss: 6.555e-07, Time: 2823.69
Epoch: 48180, Loss: 6.442e-07, Time: 2824.36
Epoch: 48190, Loss: 6.345e-07, Time: 2825.02
Epoch: 48200, Loss: 6.334e-07, Time: 2825.69
Epoch: 48210, Loss: 6.323e-07, Time: 2826.36
Epoch: 48220, Loss: 6.322e-07, Time: 2827.03
Epoch: 48230, Loss: 6.409e-07, Time: 2827.71
Epoch: 48240, Loss: 8.860e-07, Time: 2828.38
Epoch: 48250, Loss: 6.728e-07, Time: 2829.09
Epoch: 48260, Loss: 6.778e-07, Time: 2829.78
Epoch: 48270, Loss: 6.484e-07, Time: 2830.45
Epoch: 48280, Loss: 6.352e-07, Time: 2831.13
Epoch: 482

Epoch: 49900, Loss: 6.534e-07, Time: 2953.62
Epoch: 49910, Loss: 7.379e-07, Time: 2954.14
Epoch: 49920, Loss: 6.689e-07, Time: 2954.68
Epoch: 49930, Loss: 8.085e-07, Time: 2955.20
Epoch: 49940, Loss: 6.189e-07, Time: 2955.80
Epoch: 49950, Loss: 6.036e-07, Time: 2956.37
Epoch: 49960, Loss: 6.076e-07, Time: 2956.91
Epoch: 49970, Loss: 5.970e-07, Time: 2957.43
Epoch: 49980, Loss: 5.980e-07, Time: 2957.97
Epoch: 49990, Loss: 5.954e-07, Time: 2959.06


In [8]:
def ml_f(x):
    """
    Define the derivatives (RHS of the ODE) learned by ML
    """
    return np.ravel(net.predict(x.reshape(1,-1)))
    
# testing to see if it works
ml_f(gly_data[0,:])

array([ 0.89327437, -9.029732  ,  5.0351768 , -5.3296275 , -4.2815266 ,
       -2.1680076 ,  0.03295352], dtype=float32)

In [9]:
# Solve the IVP

time_points = np.arange(0, 10, step_size)

predicted_traj = odeint(lambda x, t: ml_f(x), gly_data[0,:], time_points)

In [10]:
# save the predictions for analysis in other notebooks

with open('gly_pred.npy', 'wb') as file:
    np.save(file, predicted_traj)

predicted_traj

array([[ 1.36122456,  1.4866053 ,  0.05739308, ...,  0.29558962,
         1.87303717,  0.06480158],
       [ 1.37076952,  1.40535577,  0.09402223, ...,  0.26416342,
         1.8735414 ,  0.06500119],
       [ 1.38215587,  1.33628228,  0.11167621, ...,  0.24782192,
         1.90445386,  0.06505066],
       ...,
       [-0.57196215,  3.23297342, -0.36502048, ..., -1.53900435,
         2.07049885, -1.89131542],
       [-0.58596219,  3.24259846, -0.35796348, ..., -1.53307254,
         2.09790285, -1.89438169],
       [-0.59513727,  3.23922149, -0.35071086, ..., -1.5277799 ,
         2.13665988, -1.89741348]])